In [3]:
import pandas as pd
import time
import sys
from datetime import datetime
from datetime import timedelta
from statistics import mean
from statistics import median
from math import ceil

#PARAMETERS
source = 'CarRental_LAX_2018_2019_with_Outsipp.csv'
dest = 'result_carrental_outsipp.csv'

#SOURCE             DEST
#0 COLLECT_DATE     0 COLLECT_DATE
#1 PICKUP_DATE      1 PICKUP_DATE
#2 DROP_DATE
#3 CAR_TYPE         2 CAR_TYPE
#4 LOR              3 LOR
#5 AVG_FARE         4 AVG_FARE
#6 MIN_FARE         5 MIN_FARE
#7 MAX_FARE         6 MAX_FARE
#8 MEDIAN_FARE      7 MEDIAN_FARE
#9 NO_RECORDS

#expand the data based on LOR
def distribute_lor(dfc):
    newdf = {}
    #loop for every rows
    for row in dfc:
        collect_date = row[0]
        car_type = row[3]
        #distribute based on LOR
        for k in range(row[4]):
            #pickup_date_str = (datetime.strptime(row[1], '%m/%d/%Y')+timedelta(days=k)).strftime('%m/%d/%Y')
            #add the day difference
            pickup_date_str = (datetime.strptime(row[1], '%Y-%m-%d')+timedelta(days=k)).strftime('%m/%d/%Y')
            #concatenate COLLECT DATE, THE NEW PICKUP DATE and OUTSIPP then use it as index
            index = collect_date + pickup_date_str + car_type
            if index in newdf:
                #if index already exist, then append the data into the existing list value
                (newdf[index][3]).append(row[4])
                (newdf[index][4]).append(row[5])
                (newdf[index][5]).append(row[6])
                (newdf[index][6]).append(row[7])
                (newdf[index][7]).append(row[8])
            else:
                #if index does not exist, then create a new list with with index as key
                newdf[index] = [collect_date,pickup_date_str,car_type,[row[4]],[row[5]],[row[6]],[row[7]],[row[8]]] 
    return list(newdf.values())

#start time marker
start = time.time()
#read from the input csv
df = pd.read_csv(source)
#expand the database based on the LOR
df_result = distribute_lor(df.values.tolist())
#loop over the expanded dataset and aggregate to calculate AVERAGE, MIN, MAX, MEDIAN
for row in df_result:
    row[3] = ceil(mean(row[3]))
    row[4] = round(mean(row[4]),2)
    row[5] = min(row[5])
    row[6] = max(row[6])
    row[7] = median(row[7])

#put the result into dataframe then write to an csv
df_result = pd.DataFrame(df_result, columns=['COLLECT_DATE','PICKUP_DATE','OUTSIPP' ,'LOR', 'AVG_FARE', 'MIN_FARE','MAX_FARE','MEDIAN_FARE'])
df_result.to_csv(dest, index=False)
#end time marker
end = time.time()
print('elapsed time : {0} seconds'.format(end-start))


elapsed time : 0.010535240173339844 seconds
